#### Question 9
Train An SVM classifier on the MNIST data set. Since SVM classifier are binary classifiers. You will need to use one VS the rest to classify or 10 digits. You may want to tune the hyper parameters using small validation set to speed up the process. What accuracy can you reach?

In [1]:
%pip install scikit-learn pandas numpy scikit-learn-intelex 

Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearnex import patch_sklearn
patch_sklearn()

Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [4]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

In [5]:
#Split this dataset into X_train, X_val, and X_test
X_train, X_temp, y_train, y_temp = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [6]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [7]:
# Hyperparameter tuning (using a small subset for speed)
param_grid = {'C': [0.01, 0.1, 1.0, 10.0]}
best_accuracy = 0
best_C = 0

In [ ]:
for C in param_grid['C']:
    svm = LinearSVC(random_state=42, max_iter=1000, C=C, multi_class='ovr')
    svm.fit(X_train_scaled, y_train)
    val_pred = svm.predict(X_val_scaled)
    accuracy = accuracy_score(y_val, val_pred)
    print(f'Validation accuracy for C={C}: {accuracy:.4f}')
    print(f'Predicted labels: {val_pred} vs {y_val}')
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_C = C

In [ ]:
# Train final model with best C on full training set
final_svm = LinearSVC(random_state=42, max_iter=1000, C=best_C, multi_class='ovr')
final_svm.fit(X_train_scaled, y_train)

In [ ]:
# Evaluate on test set
test_pred = final_svm.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, test_pred)

In [ ]:

print(f"Best C: {best_C}")
print(f"Validation Accuracy: {best_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")